### Para pasar las pruebas

* Primero pasar las imágenes con los parámetros que hay por defecto. 

* Luego variar los parámetros de las 3 funciones hasta obtener el mejor resultado en cada caso y el resultado final

* Documentar cómo ha variado la imagen con cada prueba(ha mejorado? ha empeorado?) y qué resultado final obtenemos(parece un dibujo?)


In [1]:
# Librerías
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import transform, exposure
from scipy import signal, sparse

In [ ]:
def obtener_trazado(img,tamkernel,num_direcciones=8,tam_masc_suavizado=5):
#Suavizado de la imagen
    img = cv2.GaussianBlur(img,(tam_masc_suavizado,tam_masc_suavizado),0)
#Cálculo del gradiente usando Sobel
    scale = 1
    delta = 0
    ddepth = cv2.CV_16S
    altura = img.shape[0] #Número de filas 
    anchura = img.shape[1] #Número de columnas


    grad_x = cv2.Sobel(img, ddepth, 1, 0, 1, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)

    grad_y = cv2.Sobel(img, ddepth, 0, 1, 1, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)


    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)

    #Obtención de la imagen gradiente final
    G = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0) #Este es el valor G en la Fig. 2 del artículo

    cv2.imshow('original', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imshow('gradiente', G)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#A partir de G calcularemos el clasificador C y posteriormente la forma en pinceladas S de nuestra imagen 
#Primero aplicaremos unas máscaras de tamaño tamkernel * 2 + 1. Esas máscaras consisten en una línea centrada
#En diferentes ángulos

    tamkernel = tamkernel * 2 + 1 
    midkernel = tamkernel // 2
    kernel = (tamkernel,tamkernel)
    kernelh = np.zeros(kernel)
    kernelh[midkernel,:] = 1 #Máscara línea horizontal
    
    #Aquí guardaremos las distintas convoluciones de las máscaras con nuestra  imagen procesada
    res_map = np.zeros((altura, anchura, num_direcciones))     
    for i in range(num_direcciones):
        ker = transform.rotate(kernelh, (180 * i)/num_direcciones)
        res_map[:,:,i] = signal.convolve2d(G,ker,mode ='same')
        
    #Aquí devuelve una matriz donde indicará el indice de la dirección donde se guarda el pixel de mayor valor
    mapa_indices_maximo_pixel = np.argmax(res_map, axis=2)

    C = np.zeros_like(res_map)

    for i in range(num_direcciones):
        #Aquí creamos las distintas clasificaciones del gradiente según la dirección que toma cada pixel
        #Guardaremos en Ci los pixeles de G en las posiciones donde nuestro mapa de direcciones tenga el valor de la direccion i
        #En otro caso será 0
        C[:,:,i] = G * (mapa_indices_maximo_pixel == i)
    
    S_prima_separado = np.zeros_like(C)

    #De nuevo volvemos a realizar la operación de convolución de nuestros Ci por la máscara de dirección i correspondiente
    #Esto dará el aspecto de pequeños trazos a los bordes de nuestra imagen
    for i in range(num_direcciones):
        ker = transform.rotate(kernelh, (i * 180) / num_direcciones)
        S_prima_separado[:,:,i] = signal.convolve2d(C[:,:,i], ker, mode='same')
    
    #Sumamos los resultados
    S_prima = np.sum(S_prima_separado, axis=2)
    
    #Normalizamos el resultado para que los pixeles se muevan en el rango[0,1]
    S_prima_normalizada = (S_prima - np.min(S_prima.ravel())) / (np.max(S_prima.ravel()) - np.min(S_prima.ravel()))
    
    #Invertimos nuestra imagen para que los trazos se vean negros sobre un fondo blanco, como si fuese un boceto
    S = 1 - S_prima_normalizada
    cv2.imshow('Trazado', S)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return S

In [ ]:
def ajustar_tonalidad(img, w_group=0,pesos_prueba=None):
    # Grupos de pesos para las diferentes tonalidades. 1ª fila tonalidad clara, 2ª tonalidad media, 3ª tonalidad oscura
    #Estos pesos son los pesos ya definidos en el artículo.
    if pesos_prueba != None:
        w = pesos_prueba
    else:
        w_mat = np.array([[11, 37, 52],
                     [29, 29, 42],
                     [2, 22, 76]])
        w = w_mat[w_group,:]
    
    #Parámetros definidos en el artículo
    
    u_b = 225
    u_a = 105
    sigma_b = 9
    mu_d = 90
    sigma_d = 11
    
    img = cv2.GaussianBlur(img,(5,5),0)
    
    # Cálculo del nuevo histograma p(v)
    num_pixel_vals = 256
    p = np.zeros(num_pixel_vals)
    for v in range(num_pixel_vals):
        p1 = (1 / sigma_b) * np.exp(-(255 - v) / sigma_b) #Función de distribución para tonos claros
        if (u_a <= v <= u_b):
            p2 = 1 / (u_b - u_a)
        else:                         #Función de distribución para tonos medios
            p2 = 0
            
            #Función de distribución para tonos oscuros
        p3 = (1 / np.sqrt(2 * np.pi * sigma_d)) * np.exp( (-np.square(v - mu_d)) / (2 * np.square(sigma_d)) )
        
        #Sumamos las distribuciones multiplicadas por los distintos pesos
        p[v] = w[0] * p1 + w[1] * p2 + w[2] * p3 * 1/100
    
    p_normalizado = p/np.sum(p)
    P = np.cumsum(p_normalizado)
    h = cv2.calcHist([img],[0],None,[256],[0,256])
    H = np.cumsum(h / np.sum(h))
    lut = np.zeros_like(p)
    for v in range(num_pixel_vals):
        dist = np.abs(P - H[v])
        argmin_dist = np.argmin(dist)
        lut[v] = argmin_dist
    lut_normalized = lut / num_pixel_vals
    J = cv2.LUT(img,lut_normalized)
    cv2.imshow('imagen original', img)
    cv2.imshow('Tonalidad Ajustada', J)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return J

In [ ]:
def obtener_textura_lapiz2(img, H, J,num_betas):
    
    altura = img.shape[0]
    anchura = img.shape[1]
    dim = (anchura,altura)
    print(img.shape)

    # Ajustamos nuestra H (textura de lápiz) y nuestra J(tonalidades) para que tengan el mismo tamaño y forma que nuestra imagen
    H_res = cv2.resize(H,dim, interpolation=cv2.INTER_CUBIC)
    print(H_res.shape)
    H_normalizada = (H_res - np.min(H_res.ravel())) / (np.max(H_res.ravel()) - np.min(H_res.ravel()))
    rows,cols = H_normalizada.shape
    n=0.7
    r=np.max(H_normalizada)
    c=r/pow(r,n)
    imgRaiz = np.zeros_like(H_normalizada)
    for i in range(rows):
        for j in range(cols):
            imgRaiz[i,j]=c*(pow(H_normalizada[i,j],n))
    
    h_res = cv2.GaussianBlur(imgRaiz,(3,3),0)
    cv2.imshow('Raiz',imgRaiz)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imshow('H_normalizada', H_normalizada)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    J_res = cv2.resize(J,dim, interpolation=cv2.INTER_CUBIC)
    cv2.imshow('J', J_res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    res_map = np.zeros((altura, anchura, num_betas))
    algo = np.ones_like(res_map)
    
    for i in range(1,num_betas):
        beta = 1 - i/num_betas
        res_map[:,:,i] = h_res**beta
        algo[:,:,i] = np.abs(J_res - res_map[:,:,i])
        
    mapa_indices = np.argmin(algo,axis=2)

    C = np.zeros_like(res_map)
    for i in range(1,num_betas):
        #Aquí creamos las distintas clasificaciones del gradiente según la dirección que toma cada pixel
        #Guardaremos en Ci los pixeles de G en las posiciones donde nuestro mapa de direcciones tenga el valor de la direccion i
        #En otro caso será 0
        C[:,:,i] = res_map[:,:,i] * (mapa_indices == i)
        
    
    T = np.sum(C, axis = 2)
    cv2.imshow('Textura', T)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return T